### Goal : Generate the files to measure x264 and x265's performances

##### Libraries

In [1]:
import os, time 
import numpy as np
import pandas as pd
from scipy import stats

## A-] Configuration options

We define the configuration options common to x264 and x265, that are influential to study software's performances

In [2]:
# common options and common values, the rest is set to default values
nb_cs = 1

## common options with arg -> replace with "--name-option arg-option"
common_arg = {
              '--aq-strength' : [str(round(i, nb_cs)) for i in np.arange(0, 3.1, 0.5)],
              '--ipratio' : [str(round(i, nb_cs)) for i in np.arange(1, 1.6, 0.1)],
              '--pbratio' : [str(round(i, nb_cs)) for i in np.arange(1, 1.5, 0.1)],
              '--psy-rd' : [str(round(i, nb_cs)) for i in np.arange(0.2, 5.1, 0.2)],
              '--qblur' : [str(round(i, nb_cs)) for i in np.arange(0.2, 0.7, 0.1)],
              '--qcomp' : [str(round(i, nb_cs)) for i in np.arange(0.6, 0.9, 0.1)],
              '--vbv-init' : [str(round(i, nb_cs)) for i in np.arange(0,1,0.1)],
              
              '--aq-mode' : [str(i) for i in range(4)],
              '--b-adapt' : [str(i) for i in range(3)],
              '--bframes' : [str(i) for i in range(0, 17, 2)],
              '--crf' : [str(i) for i in range(0, 52, 5)],
              '--keyint' : [str(i) for i in range(200, 301, 10)],
              '--lookahead-threads' : [str(i) for i in range(5)],
              '--min-keyint' : [str(i) for i in range(20, 31, 1)],
              '--qp' : [str(i) for i in range(0, 51, 10)],
              '--qpstep' : [str(i) for i in range(3, 6, 1)],
              '--qpmin' : [str(i) for i in range(5)],
              '--qpmax' : [str(i) for i in range(60, 70, 1)],
              '--rc-lookahead' : [str(i) for i in range(18, 51, 10)],
              '--ref' : [str(i) for i  in range(1, 7, 1)],
              '--vbv-bufsize' : [str(i) for i in [1000, 2000]],
    
              '--deblock' : ['-2:-2', '-1:-1', '1:1'],
              '--me' : ["dia", "hex", "umh"],
              '--overscan' : ["show", "crop"],
              '--preset' : ["ultrafast", "superfast", "veryfast", "faster", 
                           "fast", "medium", "slow", "slower", "veryslow", 
                           "placebo"],
              '--scenecut' : ['0', '10', '30', '40'],
              '--tune': ['psnr', 'ssim', 'grain', 'animation']
             }

## common options without arg -> replace with "--val-option"
common_val = {
              '--aud' : ["--aud", None],
              '--constrained-intra' : ['--constrained-intra', None],
              '--intra-refresh' : ['--intra-refresh', None],
              '--no-asm' : [None, '--no-asm'],
              '--slow-firstpass' : ['--slow-firstpass', None],
              '--weightb' : ['--weightb', '--no-weightb']
             }

# both
common_tot = common_val.copy()
common_tot.update(common_arg)

## constraints implies (feature_name1, feature_values1, feature_name2, feature_values2)
constraints =  [
                ('--scenecut', ['10', '30', '40'], '--intra-refresh', None),
                ('--ref', [str(i) for i  in range(2, 7, 1)], '--intra-refresh', None),
               ]

print("We leverage {0} common configuration options between x264 and x265"
      .format(len(common_val)+len(common_arg)))

We leverage 33 common configuration options between x264 and x265


##  B-] Input videos

We apply these configurations on input videos, so we have to define the part of the command line related to the videos

In [3]:
## Group 1

# Gaming_360P-56fe
gaming=" --input-res 640x360 --fps 24 -o ../test.mp4 ../inputs/original_videos_Gaming_360P_Gaming_360P-56fe.mkv"
# Sports_360P-4545
sports=" --input-res 624x464 --fps 25 -o ../test.mp4 ../inputs/original_videos_Sports_360P_Sports_360P-4545.mkv"


## Group 2

# Animation_480P-087e
animation=" --input-res 720x480 --fps 15.42 -o ../test.mp4 ../inputs/original_videos_Animation_480P_Animation_480P-087e.mkv"
# CoverSong_360P-5d20
cover=" --input-res 480x360 --fps 30 -o ../test.mp4 ../inputs/original_videos_CoverSong_360P_CoverSong_360P-5d20.mkv"


## Group 3

# Lecture_360P-114f
lecture=" --input-res 640x360 --fps 25 -o ../test.mp4 ../inputs/original_videos_Lecture_360P_Lecture_360P-114f.mkv"
# MusicVideo_360P-5699
music=" --input-res 640x360 --fps 30 -o ../test.mp4 ../inputs/original_videos_MusicVideo_360P_MusicVideo_360P-5699.mkv"


## Group 4

# LiveMusic_360P-1d94
live=" --input-res 640x360 --fps 25 -o ../test.mp4 ../inputs/original_videos_LiveMusic_360P_LiveMusic_360P-1d94.mkv"
# LyricVideo_360P-5e87
lyric=" --input-res 640x360 --fps 6 -o ../test.mp4 ../inputs/original_videos_LyricVideo_360P_LyricVideo_360P-5e87.mkv"

vids = [gaming, sports, animation, cover, lecture, music, live, lyric]

## C-] Software

In [4]:
softs = ['x264 ', 'x265 ']

## D-] Testing these values

>**N.B.** : to automatically test all the possible configurations, we save the logs in the logs.txt file. We then search for "[error]" in the txt file.

In [5]:
test_location = "./test/bash_x26.sh"

##### 1 - Software + Inputs

In [6]:
with open(test_location, "w") as f:
    f.write("echo 'START VIDEOS'\n")

for software in softs:
    for video in vids:
        with open(test_location, "a+") as f:
            f.write("echo 'testing "+video+"'\n")
            f.write(software+video+"\n")

with open(test_location, "a+") as f:
    f.write("echo 'END VIDEOS'\n")

##### 2 - Software + Values (configuration options without argument)

In [7]:
with open(test_location, "a+") as f:
    f.write("echo 'START VALUES OPTIONS'\n")

for software in softs:
    for feature_name in common_val.keys():
        values = common_val[feature_name]
        for val in values:
            if val:
                with open(test_location, "a+") as f:
                    f.write("echo '"+val+"'\n")
                    f.write(software+val+lyric+"\n")

with open(test_location, "a+") as f:
    f.write("echo 'END VALUES OPTIONS'\n")

##### 3 - Software + Arguments (configuration options with argument)

In [8]:
with open(test_location, "a+") as f:
    f.write("echo 'START ARGUMENTS OPTIONS'\n")

for software in softs:
    for feature_name in common_arg.keys():
        args = common_arg[feature_name]
        for argument in args:
            with open(test_location, "a+") as f:
                f.write("echo '"+feature_name+" "+argument+"'\n")
                f.write(software+feature_name+' '+argument+lyric+"\n")

with open(test_location, "a+") as f:
    f.write("echo 'END ARGUMENTS OPTIONS'\n")

>Bash command, **to run in command line once in the test folder** : ``bash bash_x26.sh &> logs.txt``

Seems to be ok, but few warnings are raised:
- ``x265 [warning]: Max References > 1 + intra-refresh is not supported , setting max num references = 1`` -> ok
- ``x265 [warning]: B pyramid cannot be enabled when max references is 1, Disabling B pyramid`` -> default
- ``x265 [warning]: Open Gop disabled, Intra Refresh is not compatible with openGop`` -> default
- ``x265 [warning]: Scenecut is disabled when Intra Refresh is enabled`` -> ok
- ``x265 [warning]: Source height < 720p; disabling lookahead-slices`` -> default

## E-] Generating the configurations

### 1 - Generation 

> Memo : suppress duplicate lines, limit to 50 k configs

#### a-] Selecting feature values

We used random sampling to generate the configurations, w.r.t the constraints.

In [9]:
def gen_random_config():
    """ generates a dict of random (i.e. uniform selection 
        of values) configuration working for x264 and x265 """
    global common_val, common_arg, constraints
    
    cmd_line_args = dict()

    # values and boolean
    for feature_name in common_val:
        values = common_val[feature_name]
        val = values[np.random.randint(len(values))]
        if val:
            cmd_line_args[feature_name] = val

    # features with arguments
    for feature_name in common_arg:
        args = common_arg[feature_name]
        arg = args[np.random.randint(len(args))]
        cmd_line_args[feature_name] = arg

    # constraints
    for cs in constraints:
        for cmd_name in cmd_line_args:
            cmd = cmd_line_args[cmd_name]
            if cs[0]==cmd_name and cmd in cs[1]:
                if cs[2] in cmd_line_args:
                    #print("{0} changed to {1}".format(cs[2],cs[3]))
                    cmd_line_args[cs[2]] = cs[3]

    return cmd_line_args

gen_random_config()

{'--aud': '--aud',
 '--constrained-intra': '--constrained-intra',
 '--intra-refresh': None,
 '--weightb': '--weightb',
 '--aq-strength': '1.5',
 '--ipratio': '1.5',
 '--pbratio': '1.2',
 '--psy-rd': '3.4',
 '--qblur': '0.2',
 '--qcomp': '0.6',
 '--vbv-init': '0.1',
 '--aq-mode': '0',
 '--b-adapt': '2',
 '--bframes': '12',
 '--crf': '25',
 '--keyint': '300',
 '--lookahead-threads': '1',
 '--min-keyint': '24',
 '--qp': '50',
 '--qpstep': '4',
 '--qpmin': '2',
 '--qpmax': '63',
 '--rc-lookahead': '18',
 '--ref': '3',
 '--vbv-bufsize': '1000',
 '--deblock': '1:1',
 '--me': 'umh',
 '--overscan': 'crop',
 '--preset': 'medium',
 '--scenecut': '10',
 '--tune': 'psnr'}

#### b-] Generation of the corresponding command lines...

In [10]:
def dict2cmd_line(software, features, video):
    """ converts the list of feature arguments into a working command line """
    # inputs :
    # - software, e.g. x264 or x265, see C-]
    # - features, a dict of configuration options, see A-]
    # - video, see B-]
    # output :  a command line launching the encoding of the input video with the chosen sotfware
    
    cmd_line = str(software)
    
    for f_name in features:
        cmd_line+=" "
        if f_name in common_val:
            if features[f_name]:
                cmd_line+=features[f_name]
        elif f_name in common_arg:
            cmd_line+=f_name+" "+features[f_name]
            
    cmd_line+=" "+video
    
    return cmd_line

dict2cmd_line("x264", gen_random_config(), lyric)

'x264 --no-weightb --aq-strength 1.0 --ipratio 1.3 --pbratio 1.4 --psy-rd 4.2 --qblur 0.6 --qcomp 0.8 --vbv-init 0.2 --aq-mode 2 --b-adapt 2 --bframes 12 --crf 15 --keyint 300 --lookahead-threads 3 --min-keyint 22 --qp 20 --qpstep 5 --qpmin 1 --qpmax 65 --rc-lookahead 38 --ref 2 --vbv-bufsize 2000 --deblock -2:-2 --me hex --overscan show --preset faster --scenecut 0 --tune grain  --input-res 640x360 --fps 6 -o ../test.mp4 ../inputs/original_videos_LyricVideo_360P_LyricVideo_360P-5e87.mkv'

#### c-] ... and save it in a bash file

In [11]:
with open("./test/inferC_loop.sh", "w") as f:
    f.write("echo 'START TESTING OPTIONS'\n")
    for software in softs:
        for i in range(50):
            f.write(dict2cmd_line(software, gen_random_config(), lyric)+"\n")

### 2 - Infer constraints from logs

Run the bash file with the following command:

>Bash command, **to run in command line once in the test subfolder** : ``bash inferC_loop.sh &> logs_inferC.txt``

The "infer constraints" loop :

-> test new configs
-> find new constraints
-> add constraints to the generation process

-> test new configs w.r.t. the constraints
-> find new constraints
-> add constraints to the generation process

-> etc.

#### Results:

New errors:
- ``x265 [error]: Lookahead depth must be greater than the max consecutive bframe count`` -> chosen solution : increase rc-lookahead minimum value (10 -> 18) 

New warnings:
- ``x265 [warning]: VBV is incompatible with constant QP, ignored.`` -> managed by x264 & x265

### 3 - Generate the final configuration space

### a -] Dataset

Final csv

In [19]:
names = list(common_tot.keys())
list_conf = []
nbconf = 3125

while len(list_conf) < nbconf:
    rand_conf = gen_random_config()
    conf_values = [None]*len(names)
    for i in range(len(names)):
        n = names[i]
        if n in rand_conf:
            conf_values[i] = rand_conf[n]
    if conf_values not in list_conf:
        list_conf.append(conf_values)

final_conf = pd.DataFrame(list_conf, columns = names)
final_conf

,--aud,--constrained-intra,--intra-refresh,--no-asm,--slow-firstpass,--weightb,--aq-strength,--ipratio,--pbratio,--psy-rd,...,--qpmax,--rc-lookahead,--ref,--vbv-bufsize,--deblock,--me,--overscan,--preset,--scenecut,--tune
0,None,--constrained-intra,None,None,None,--weightb,0.0,1.0,1.0,1.8,...,67,28,4,1000,-2:-2,umh,crop,veryslow,30,ssim
1,--aud,None,None,None,None,--no-weightb,0.0,1.1,1.2,0.4,...,66,18,3,1000,1:1,hex,crop,fast,0,grain
2,None,None,None,None,None,--no-weightb,3.0,1.2,1.0,1.8,...,63,48,1,1000,-2:-2,dia,crop,veryslow,40,grain
3,--aud,None,None,None,--slow-firstpass,--no-weightb,0.0,1.5,1.0,2.0,...,62,38,1,1000,-2:-2,umh,crop,faster,40,grain
4,None,--constrained-intra,None,--no-asm,None,--no-weightb,1.0,1.6,1.3,4.4,...,66,28,3,1000,-2:-2,dia,show,superfast,30,animation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3120,None,None,None,--no-asm,None,--no-weightb,1.5,1.2,1.1,0.6,...,64,28,1,2000,-1:-1,dia,crop,slow,40,psnr
3121,--aud,--constrained-intra,None,None,None,--weightb,3.0,1.3,1.3,3.2,...,62,18,4,1000,1:1,hex,show,ultrafast,0,ssim
3122,--aud,None,None,None,--slow-firstpass,--weightb,2.0,1.2,1.1,3.6,...,69,18,1,1000,1:1,hex,crop,superfast,10,grain
3123,None,None,None,None,None,--no-weightb,0.5,1.5,1.0,1.0,...,61,38,6,2000,-1:-1,umh,show,superfast,30,grain


In [21]:
# final_conf.to_csv("configs_x26x.csv")

### b-] Bash files

##### Import data

In [22]:
x26x_config = pd.read_csv("configs_x26x.csv", delimiter=',', index_col = 0)

option_names =  x26x_config.columns

x26x_config = x26x_config.fillna('None')

nb_config = x26x_config.shape[0]

x26x_config

,--aud,--constrained-intra,--intra-refresh,--no-asm,--slow-firstpass,--weightb,--aq-strength,--ipratio,--pbratio,--psy-rd,...,--qpmax,--rc-lookahead,--ref,--vbv-bufsize,--deblock,--me,--overscan,--preset,--scenecut,--tune
0,None,--constrained-intra,None,None,None,--weightb,0.0,1.0,1.0,1.8,...,67,28,4,1000,-2:-2,umh,crop,veryslow,30,ssim
1,--aud,None,None,None,None,--no-weightb,0.0,1.1,1.2,0.4,...,66,18,3,1000,1:1,hex,crop,fast,0,grain
2,None,None,None,None,None,--no-weightb,3.0,1.2,1.0,1.8,...,63,48,1,1000,-2:-2,dia,crop,veryslow,40,grain
3,--aud,None,None,None,--slow-firstpass,--no-weightb,0.0,1.5,1.0,2.0,...,62,38,1,1000,-2:-2,umh,crop,faster,40,grain
4,None,--constrained-intra,None,--no-asm,None,--no-weightb,1.0,1.6,1.3,4.4,...,66,28,3,1000,-2:-2,dia,show,superfast,30,animation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3120,None,None,None,--no-asm,None,--no-weightb,1.5,1.2,1.1,0.6,...,64,28,1,2000,-1:-1,dia,crop,slow,40,psnr
3121,--aud,--constrained-intra,None,None,None,--weightb,3.0,1.3,1.3,3.2,...,62,18,4,1000,1:1,hex,show,ultrafast,0,ssim
3122,--aud,None,None,None,--slow-firstpass,--weightb,2.0,1.2,1.1,3.6,...,69,18,1,1000,1:1,hex,crop,superfast,10,grain
3123,None,None,None,None,None,--no-weightb,0.5,1.5,1.0,1.0,...,61,38,6,2000,-1:-1,umh,show,superfast,30,grain


Since x264 and x265 logs differ, we have to slightly modify the related bash files.

##### Scripts x264

In [23]:
cmd_x26x = []
csvLines = []

for j in range(nb_config):
    l = x26x_config.iloc[j]
    csvLine = 'csvLine="$numb,'

    x26x_line = '{ time x264 '
    for i in range(x26x_config.shape[1]):
        arg = l[i]
        if arg!='None':
            if i >= 6:
                x26x_line+=' '+str(option_names[i])
            x26x_line+=' '+str(arg)
        csvLine+=str(arg)+','
    x26x_line+=' --output $outputlocation $inputconf $inputlocation ; } 2> $logfilename'
    cmd_x26x.append(x26x_line)
    csvLine+='"'
    csvLines.append(csvLine)

for i in range(nb_config):
    with open('./scripts/x264/'+str(i)+'.sh','w') as f:
        f.write('#!/bin/sh\n\n')
        f.write("numb='"+str(i+1)+"'")
        f.write('\nlogfilename="./logs/$numb.log"\ninputlocation="$1"\noutputlocation="test.mp4"')
        f.write('\ninputconf="$2"\n\n')
        f.write(cmd_x26x[i])
        f.write('\n# extract output video size\n')
        f.write("size=`ls -lrt $outputlocation | awk '{print $5}'`\n")
        f.write('# analyze log to extract relevant timing information and CPU usage\n')
        f.write("""time=`grep "user" $logfilename | sed 's/,/./; s/elapsed/,/; s/system/,/; s/user/,/; s/ //g' | cut -d "%" -f 1`""")
        f.write('\n# analyze log to extract fps and kbs\n')
        f.write("""persec=`grep "encoded" $logfilename | sed 's/encoded\(.*\)frames// ; s/fps// ; s//,/; s/ //g' | cut -d "k" -f 1`""")
        f.write('\n# clean\n#rm $outputlocation\n\n')
        f.write(csvLines[i])
        f.write('\ncsvLine="$csvLine$size,$time,$persec"\necho "$csvLine"')

#### Scripts x265

In [24]:
cmd_x26x = []
csvLines = []

for j in range(nb_config):
    l = x26x_config.iloc[j]
    csvLine = 'csvLine="$numb,'

    x26x_line = '{ time x265 '
    for i in range(x26x_config.shape[1]):
        arg = l[i]
        if arg!='None':
            if i >= 6:
                x26x_line+=' '+str(option_names[i])
            x26x_line+=' '+str(arg)
        csvLine+=str(arg)+','
    x26x_line+=' --output $outputlocation $inputconf $inputlocation ; } 2> $logfilename'
    cmd_x26x.append(x26x_line)
    csvLine+='"'
    csvLines.append(csvLine)

for i in range(nb_config):
    with open('./scripts/x265/'+str(i)+'.sh','w') as f:
        f.write('#!/bin/sh\n\n')
        f.write("numb='"+str(i+1)+"'")
        f.write('\nlogfilename="./logs/$numb.log"\ninputlocation="$1"\noutputlocation="test.mp4"')
        f.write('\ninputconf="$2"\n\n')
        f.write(cmd_x26x[i])
        f.write('\n# extract output video size\n')
        f.write("size=`ls -lrt $outputlocation | awk '{print $5}'`\n")
        f.write('# analyze log to extract relevant timing information and CPU usage\n')
        f.write("""time=`grep "user" $logfilename | sed 's/,/./; s/elapsed/,/; s/system/,/; s/user/,/; s/ //g' | cut -d "%" -f 1`""")
        f.write('\n# analyze log to extract fps and kbs\n')
        f.write("""persec=`grep "encoded" $logfilename | sed 's/encoded\(.*\)(//; s/encoded// ; s/fps)// ; s//,/; s/ //g' | cut -d "k" -f 1`""")
        f.write('\n# clean\n#rm $outputlocation\n\n')
        f.write(csvLines[i])
        f.write('\ncsvLine="$csvLine$size,$time,$persec"\necho "$csvLine"')

Test the bash files:

``bash ./scripts/x265/0.sh ./inputs/original_videos_MusicVideo_360P_MusicVideo_360P-5699.mkv "--input-res 640x360 --fps 30"``

``bash ./scripts/x264/0.sh ./inputs/original_videos_MusicVideo_360P_MusicVideo_360P-5699.mkv "--input-res 640x360 --fps 30"``

For an example, here are the occurences of the ``--preset`` values:

### 4-] Validation: check the sampling with statistical tests

#### a-] Idea

Check that the generated configuration accurately represent all values.

In [25]:
pd.Series(x26x_config["--preset"]).value_counts()

superfast    338
fast         338
slow         335
veryfast     323
medium       316
faster       309
veryslow     305
placebo      288
ultrafast    287
slower       286
Name: --preset, dtype: int64

#### b-] Kolmogorov-Smirnov test

We used one-sample Kolmogorov-Smirnov tests to compare the cumulative distributive function (aka cdf) of the generated distribution to the theorical uniform distribution.

The idea of this test is to study the maximal difference of two cdfs, (i.e. $max_{x}|F_{1}(x) - F_{2}(x)|$, where $F_{1}$ and $F_{2}$ are the two cdfs). The bigger the difference, the more the distributions are far from each other.

The one-sample version of this test compares the empirical cdf ($F_{1} =  F_{emp}$) to the theorical cdf ($F_{2} =  F_{t}$). Here, we want the theorical law to be uniform. Let us say that an option has ten ordered values $ k = 1 ... 10$. Then, the empirical cdf respects $\forall k \in [|1,10|], F_{emp}(k) =  c(k)*\frac{1}{\#(config)}$, where c(k) is the count of ``k`` value occurences, while the theorical cdf respects $\forall k \in [|1,10|], F_{t}(k) = P(X<=k) = \frac{k}{10}$.

We choose the threshold $0.05$. The null hypothesis of same distribution (i.e. the sampling can be considered as uniform) is rejected if $pval < 0.05$.

#### c-] Results

In [26]:
for opt_name in option_names:
    opt_values_count = pd.Series(x26x_config[opt_name]).value_counts()
    opt_values_count /= nb_config
    cdf_values = []
    s = 0
    for opt_val_count in opt_values_count:
        s+=opt_val_count
        cdf_values.append(s)
    cdf_values.append(1)
    pval = stats.kstest(cdf_values, 'uniform').pvalue
    print(opt_name, ", p-val = ", str(round(pval,3))+",", (pval>=0.05)*"Uniform sampling")

--aud , p-val =  0.074, Uniform sampling
--constrained-intra , p-val =  0.074, Uniform sampling
--intra-refresh , p-val =  0.0, 
--no-asm , p-val =  0.074, Uniform sampling
--slow-firstpass , p-val =  0.074, Uniform sampling
--weightb , p-val =  0.074, Uniform sampling
--aq-strength , p-val =  0.629, Uniform sampling
--ipratio , p-val =  0.629, Uniform sampling
--pbratio , p-val =  0.426, Uniform sampling
--psy-rd , p-val =  0.984, Uniform sampling
--qblur , p-val =  0.426, Uniform sampling
--qcomp , p-val =  0.309, Uniform sampling
--vbv-init , p-val =  0.86, Uniform sampling
--aq-mode , p-val =  0.309, Uniform sampling
--b-adapt , p-val =  0.188, Uniform sampling
--bframes , p-val =  0.819, Uniform sampling
--crf , p-val =  0.886, Uniform sampling
--keyint , p-val =  0.893, Uniform sampling
--lookahead-threads , p-val =  0.426, Uniform sampling
--min-keyint , p-val =  0.867, Uniform sampling
--qp , p-val =  0.533, Uniform sampling
--qpstep , p-val =  0.188, Uniform sampling
--qpmin ,

/home/llesoil/anaconda3/envs/x264/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  # This is added back by InteractiveShellApp.init_path()


Except for ``--intra-refresh`` (due to many constraints, like ``--intra-refresh`` activated implies ``--ref`` =1), all features can be considered as uniformly distributed with the threshold $0.05$